In [1]:
import pandas as pd
import numpy as np
import os, sys, re

In [5]:
import os
fn = r"/Users/dblyon/Downloads/files2convert_clean_RJC.txt"
# fn = r"/Users/dblyon/modules/cpr/metaprot/conf/files_2_copy_Win_virtual_server.txt"
fn_out = r'/Users/dblyon/modules/cpr/metaprot/conf/files_2_copy_Win_virtual_server_2.txt'
prefix = r"/biologic/CPR/cprarchive/Proteomics/" # /2015/01/QE3/FileName.raw
with open(fn, 'r') as fh:
    with open(fn_out, 'w') as fh_out:
        for line in fh:
            line = line.strip().replace("\\", "/").replace("Z:/MSDATA/", "/").replace("V:/MSDATA/", "/")[1:]
            line2write = os.path.join(prefix, line)
            print line2write
            fh_out.write(line2write + "\n")

/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_448.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_449.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_450.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_451.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_452.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_453.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_454.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_455.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_456.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE3/20141015_QE3_UPLC8_RJC_SA_KU16_457.raw
/biologic/CPR/cprarchive/Proteomics/2014/10/QE5/20141027_QE5_UPLC9_RJC_SA_KU16_458.raw
/biologic/CPR/cprarchive/Proteomics/2014/10

In [ ]:
import os, sys
from subprocess import call

dir_jvo = r"Z:\MSDATA"
with open(file_list_txt, "r") as fh:
    for line in fh:
        fn = line.strip()[1:]
        fn_abs = os.path.join(dir_jvo, fn)
        if os.path.exits(fn_abs):
            print fn_abs
    

          
file_list_txt = r"Z:\Lyon\MS_data\filesList_RAW_KU16.txt"
config_txt = r"Z:\Lyon\MS_data\MGF\config.txt"
output_dir = r"Z:\Lyon\MS_data\MGF"
shellcmd = "msconvert -f {} -c {} -o {}".format(file_list_txt, config_txt, output_dir)
#call(shellcmd, shell=True)
print shellcmd          

In [11]:
sys.path.append("./static/python/")

In [12]:
import userinput

In [32]:
reload(userinput)

<module 'userinput' from './static/python/userinput.py'>

In [37]:
from StringIO import StringIO

In [48]:
import itertools

In [67]:
s = StringIO()

In [68]:
f = u'BUBU;BUBU2;BUBU3\r\nAUBU;AUBU2;AUBU3\r\nP40545\r\nP09201\r\nP14066\r\nP25040\r\nP20050\r\nP40327\r\nQ01939\r\nQ01939'
b = u'BUBU\t\r\nBUBU\t\r\nP00359\t11,07751325\r\nP00925\t10,96978388\r\nP02994\t10,76849751\r\nP06169\t10,74716312\r\nP00560\t10,74614359\r\nP00950\t10,74149039\r\nP00549\t10,70474233\r\nP00330\t10,6690378\r\nAIAIAI\r\nUUUU;IIII'

In [69]:
# zip(f.split("\r\n"), b.split("\r\n"))

In [70]:
s = StringIO()
s.write('foreground\tbackground\tintensity\r')
for a, b in itertools.izip_longest(f.split("\r\n"), b.split("\r\n"), fillvalue="\t"):
    s.write(a + "\t" + b.strip() + "\r")

In [75]:
s.tell()

0

In [80]:
s.seek(0)

In [81]:
s.tell()

0

In [82]:
set(["all_GO", "BP", "CP", "MF"])

{'BP', 'CP', 'MF', 'all_GO'}

In [84]:
{"BP", "CP", "MF"}

{'BP', 'CP', 'MF'}

In [78]:
s.read()

u'foreground\tbackground\tintensity\rBUBU;BUBU2;BUBU3\tBUBU\rAUBU;AUBU2;AUBU3\tBUBU\rP40545\tP00359\t11,07751325\rP09201\tP00925\t10,96978388\rP14066\tP02994\t10,76849751\rP25040\tP06169\t10,74716312\rP20050\tP00560\t10,74614359\rP40327\tP00950\t10,74149039\rQ01939\tP00549\t10,70474233\rQ01939\tP00330\t10,6690378\r\t\tAIAIAI\r\t\tUUUU;IIII\r'

In [59]:
s.split("\r")

[u'foreground\tbackground\tintensity',
 u'BUBU;BUBU2;BUBU3\tBUBU',
 u'AUBU;AUBU2;AUBU3\tBUBU',
 u'P40545\tP00359\t11,07751325',
 u'P09201\tP00925\t10,96978388',
 u'P14066\tP02994\t10,76849751',
 u'P25040\tP06169\t10,74716312',
 u'P20050\tP00560\t10,74614359',
 u'P40327\tP00950\t10,74149039',
 u'Q01939\tP00549\t10,70474233',
 u'Q01939\tP00330\t10,6690378',
 u'\t\tAIAIAI',
 u'\t\tUUUU;IIII',
 u'']

In [54]:
x = 'intensity\tbackground\tforeground\r\tBUBU\t"BUBU;BUBU2;BUBU3"\r\tBUBU\tAUBU;AUBU2;AUBU3\r11,07751325\tP00359\tP40545\r10,96978388\tP00925\tP09201\r10,76849751\tP02994\tP14066\r10,74716312\tP06169\tP25040\r10,74614359\tP00560\tP20050\r10,74149039\tP00950\tP40327\r10,70474233\tP00549\tQ01939\r10,6690378\tP00330\tQ01939\r10,64206863\tP05743\tP38142\r10,64061061\t"P02407;P14127"\tQ01939\r10,63619693\t"P0C2H6;P0C2H7"\tP53549\r10,6231769\t"P35997;P38711"\tP53549\r10,61589747\tP29453\tP39959\r10,59787062\tO14455\tP38787\r10,59595893\tP04456\tP38787\r10,5819724\tP14540\tP12684\r10,58108453\t"P0CX84;P0CX85"\tP12684\r10,58103895\tP05755\tP38122\r10,56810781\t"P0CX31;P0CX32"\tP38122\r10,54364595\t"P0CX55;P0CX56"\tP0CX47\r10,5379072\tP38061'

In [55]:
x.split("\r")

['intensity\tbackground\tforeground',
 '\tBUBU\t"BUBU;BUBU2;BUBU3"',
 '\tBUBU\tAUBU;AUBU2;AUBU3',
 '11,07751325\tP00359\tP40545',
 '10,96978388\tP00925\tP09201',
 '10,76849751\tP02994\tP14066',
 '10,74716312\tP06169\tP25040',
 '10,74614359\tP00560\tP20050',
 '10,74149039\tP00950\tP40327',
 '10,70474233\tP00549\tQ01939',
 '10,6690378\tP00330\tQ01939',
 '10,64206863\tP05743\tP38142',
 '10,64061061\t"P02407;P14127"\tQ01939',
 '10,63619693\t"P0C2H6;P0C2H7"\tP53549',
 '10,6231769\t"P35997;P38711"\tP53549',
 '10,61589747\tP29453\tP39959',
 '10,59787062\tO14455\tP38787',
 '10,59595893\tP04456\tP38787',
 '10,5819724\tP14540\tP12684',
 '10,58108453\t"P0CX84;P0CX85"\tP12684',
 '10,58103895\tP05755\tP38122',
 '10,56810781\t"P0CX31;P0CX32"\tP38122',
 '10,54364595\t"P0CX55;P0CX56"\tP0CX47',
 '10,5379072\tP38061']

In [39]:
s.write("bubu")

In [40]:
s.tell()

4

In [33]:
fn = r"/Users/dblyon/CloudStation/CPR/BTW_GO/UserInput.txt"
fn = r"/Users/dbl/modules/cpr/agotool/static/data/exampledata/exampledata_yeast.txt"
# fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/refactorUserinput.txt"
df = pd.read_csv(fn, sep='\t', decimal=",")#, converters={"background_int": float, "background_an": str, "foreground_an": str})

In [34]:
# fh = open(fn, "r").read()
ui = userinput.Userinput(fn=fn, foreground_string=None, background_string=None,
col_foreground='foreground', col_background='background', col_intensity='intensity',
num_bins=100, decimal='.', method="abundance_correction")

In [35]:
ui.background

,background,intensity
0,BUBU,-1.000000
2,P00359,11.077513
3,P00925,10.969784
4,P02994,10.768498
5,P06169,10.747163
6,P00560,10.746144
7,P00950,10.741490
8,P00549,10.704742
9,P00330,10.669038
10,P05743,10.642069


In [314]:
cond = ui.foreground["intensity"] > -1
bins = pd.cut(ui.foreground.loc[cond, "intensity"], bins=100, retbins=True)[1]
bins = [-1] + list(bins)

In [315]:
ui.foreground

,foreground,intensity
0,P00549,1.700000
1,AN3,1.700000
2,P00330,2.600000
3,P05743,3.642069
4,P02407,4.640611
5,P0C2H7,5.636197
6,P35997;P38711,7.623177
7,YOYOYO,-1.000000


In [26]:
for x in ui.iter_bins():
#     print wf, bins
#     print bg, fg
#     print ans, cf
    print x
    print

In [323]:
mbps = 30.0
gb = mbps / 1000.0
min_ = gb * 60
hour = min_ * 60
print min_
print hour
tb = 2.0
(tb * 1000) / hour

1.8
108.0


18.518518518518523

In [157]:
bins = pd.cut(ui.foreground["intensity"], bins=100, retbins=True)[1]
groups_fg = ui.foreground.groupby(pd.cut(ui.foreground["intensity"], bins=bins))
groups_bg = ui.background.groupby(pd.cut(ui.background["intensity"], bins=bins))

In [160]:
# ui.background[ui.background["intensity"] <= 0]

In [159]:
for group_fg, group_bg in zip(groups_fg, groups_bg):
    bins_in_group = group[0]
    pg_fg = group_fg[1]["foreground"]
    pg_bg = group_bg[1]["background"]
    print pg_fg, len(pg_fg)
    print pg_bg, len(pg_bg)
    print 
    break

0       BUBU;BUBU2;BUBU3
1       AUBU;AUBU2;AUBU3
3                 P09201
4                 P14066
6                 P20050
9                 P38142
11                P39959
16                Q03868
17                Q04004
18                Q04383
20                Q06592
23                Q12348
110               Q2V2Q1
112               Q08981
145               P51979
212               P38806
411               P07261
446               Q05471
483               P40963
574               P38157
702               P32833
787               P38282
806               P50104
981               Q12247
992               P40316
1029              P31111
1046              P38140
1055              P53718
1079              P32389
1109              Q12472
1119              Q9URQ3
1123              P18412
Name: foreground, dtype: object 32
0    BUBU
Name: background, dtype: object 1



In [116]:
# for group in groups:
# #     bins_in_group = group[0]
# #     pg_in_group = group[1]["foreground"]
# #     print bins_in_group
# #     print pg_in_group
#     print group
#     print 

In [80]:
hist, bins = np.histogram(ui.foreground["intensity"], bins=100)

In [82]:
bins # (-1.0121, -0.879]

array([ -1.        ,  -0.87922487,  -0.75844974,  -0.6376746 ,
        -0.51689947,  -0.39612434,  -0.2753492 ,  -0.15457407,
        -0.03379894,   0.08697619,   0.20775133,   0.32852646,
         0.44930159,   0.57007672,   0.69085186,   0.81162699,
         0.93240212,   1.05317725,   1.17395239,   1.29472752,
         1.41550265,   1.53627778,   1.65705291,   1.77782805,
         1.89860318,   2.01937831,   2.14015345,   2.26092858,
         2.38170371,   2.50247884,   2.62325398,   2.74402911,
         2.86480424,   2.98557937,   3.10635451,   3.22712964,
         3.34790477,   3.4686799 ,   3.58945504,   3.71023017,
         3.8310053 ,   3.95178043,   4.07255557,   4.1933307 ,
         4.31410583,   4.43488096,   4.5556561 ,   4.67643123,
         4.79720636,   4.91798149,   5.03875663,   5.15953176,
         5.28030689,   5.40108202,   5.52185716,   5.64263229,
         5.76340742,   5.88418255,   6.00495769,   6.12573282,
         6.24650795,   6.36728308,   6.48805822,   6.60

In [85]:
hist

array([30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
        0,  1,  0,  0,  1,  1,  1,  3,  0,  5,  5,  6,  7, 12, 14, 19, 15,
       17, 19, 25, 28, 21, 36, 33, 44, 47, 61, 57, 59, 48, 42, 51, 63, 46,
       45, 54, 46, 40, 28, 18, 18, 21, 15, 26, 16,  7,  4,  0,  2])

In [83]:
10.71518785 -  10.83596299

-0.120775140000001

In [84]:
9.62821166 -   9.74898679

-0.12077513000000017

In [ ]:
df.loc[0, "background_int"]

In [ ]:
# np.histogram(df["background_int"], bins=10)
np.histogram(df.loc[pd.notnull(df["background_int"]), "background_int"], bins=10)

In [ ]:
ui.background_df

In [ ]:
# import math

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# arr = list(np.arange(10))
# arr += [2.3]*8
# hist, bin_edges = np.histogram(arr, bins=10, density=False)
# plt.hist(arr, bins=bin_edges)  # plt.hist passes it's arguments to np.histogram
# plt.show()

In [ ]:
# good stuff

In [ ]:
# foreground proteinGroups with intensities
intensity_foreground = []
for proteinGroup in ui.foreground_ser:
    an_first_in_proteinGroup = proteinGroup.split(";")[0]
    intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     try:      
#         intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     except KeyError:
#         # can't occur, since defaultdict
#         raise StopIteration

In [ ]:
ui.foreground_df = pd.DataFrame(data=intensity_foreground, columns=["Int"])
ui.foreground_df["AN"] = ui.foreground_ser

In [ ]:
df = ui.foreground_df
df

In [ ]:
1.5*30

In [ ]:
# create bins
# which proteinGroups from background are within bin

In [ ]:
num_bins = 100
bins = np.linspace(df["Int"].min(), df["Int"].max(), num_bins)
groups = df.groupby(pd.cut(df["Int"], bins))

In [ ]:
# for group in groups:
#     print group
#     print